# IMPORT LIBRARIES

In [24]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import urllib



# GET DATASET IN DATAFRAME

In [25]:
df_onlineretail = pd.read_csv('retail.csv')

# ANALYSING THE RAW DATASET

In [26]:
df_onlineretail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [27]:
df_onlineretail.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [28]:
df_onlineretail.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

# DATA CLEANING FIX 1 - Dropping rows without Customer ID

In [29]:

df_onlineretail = df_onlineretail.dropna(subset=['Customer ID'])

In [46]:
df_onlineretail.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [47]:
 df_onlineretail.drop_duplicates()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France


In [48]:
df_onlineretail = df_onlineretail.drop_duplicates()

In [49]:
df_onlineretail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 797885 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      797885 non-null  object        
 1   StockCode    797885 non-null  object        
 2   Description  797885 non-null  object        
 3   Quantity     797885 non-null  int64         
 4   InvoiceDate  797885 non-null  datetime64[ns]
 5   Price        797885 non-null  float64       
 6   Customer ID  797885 non-null  int64         
 7   Country      797885 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 54.8+ MB


# DATA CLEANING FIX 2 - Convert Datatype

In [50]:
# Floats use more space and are slightly slower than Integers. 
#   Converting to integer saves memory and speeds up operations.
df_onlineretail['Customer ID'] = df_onlineretail['Customer ID'].astype(int)

# function has intelligence to handle date format, timezones, and manages errors (like leap years)
df_onlineretail['InvoiceDate'] = pd.to_datetime(df_onlineretail['InvoiceDate'])

In [51]:
df_onlineretail['Customer ID'].dtype

dtype('int64')

In [52]:
df_onlineretail['InvoiceDate'].info()

<class 'pandas.core.series.Series'>
Index: 797885 entries, 0 to 1067370
Series name: InvoiceDate
Non-Null Count   Dtype         
--------------   -----         
797885 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 12.2 MB


# DATA CLEANING FIX 3 - Create Revenue Attribute

In [53]:
df_onlineretail['Revenue'] = df_onlineretail['Quantity'] * df_onlineretail['Price']
df_onlineretail.head()  

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


# DATA CLEANING FIX 4 - Filtering valid sales only in new df

In [54]:
# Create a new DataFrame which contains only valid sales records that have positive Quantity and Price 
df_sales = df_onlineretail[(df_onlineretail['Quantity'] > 0) & (df_onlineretail['Price'] > 0)]

# DATA CLEANING FIX 5 - Duplicate records

In [55]:
df_sales.duplicated().sum()

np.int64(0)

In [56]:
df_sales.shape

(779425, 9)

In [22]:
df_sales.drop_duplicates

<bound method DataFrame.drop_duplicates of         Invoice StockCode                          Description  Quantity  \
0        489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1        489434    79323P                   PINK CHERRY LIGHTS        12   
2        489434    79323W                  WHITE CHERRY LIGHTS        12   
3        489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4        489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
...         ...       ...                                  ...       ...   
1067366  581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
1067367  581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
1067368  581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
1067369  581587     22138        BAKING SET 9 PIECE RETROSPOT          3   
1067370  581587      POST                              POSTAGE         1   

                InvoiceDate  Price  Customer

# STATISTICAL ANALYSIS

In [13]:
df_sales.describe()

,Quantity,InvoiceDate,Price,Customer ID,Revenue
count,805549.000000,805549,805549.000000,805549.000000,805549.000000
mean,13.290522,2011-01-02 10:24:44.106814464,3.206561,15331.954970,22.026505
min,1.000000,2009-12-01 07:45:00,0.001000,12346.000000,0.001000
25%,2.000000,2010-07-07 12:08:00,1.250000,13982.000000,4.950000
50%,5.000000,2010-12-03 15:10:00,1.950000,15271.000000,11.850000
75%,12.000000,2011-07-28 13:05:00,3.750000,16805.000000,19.500000
max,80995.000000,2011-12-09 12:50:00,10953.500000,18287.000000,168469.600000
std,143.634088,NaN,29.199173,1696.737039,224.041928


In [57]:
df_sales.shape

(779425, 9)

#  SQL CONNECTION

In [58]:
pip install pandas sqlalchemy pyodbc


Note: you may need to restart the kernel to use updated packages.


In [59]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 18 for SQL Server']


In [60]:
params = urllib.parse.quote_plus(
    "DRIVER=ODBC Driver 18 for SQL Server;"
    "SERVER=ANIRUDH\\SQLEXPRESS;"
    "DATABASE=OnlineRetail;"
    "Trusted_Connection=yes;"
    "Encrypt=yes;"
    "TrustServerCertificate=yes;"
)

engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={params}",
    fast_executemany=True
)


# TEST CONNECTION

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(
        text("SELECT @@SERVERNAME AS server_name, DB_NAME() AS database_name")
    )
    print(result.fetchone())


('Anirudh\\SQLEXPRESS', 'OnlineRetail')


# LOAD DATA TO SQL

In [62]:
df_sales.to_sql(
    "fact_sales_raw",
    engine,
    if_exists="replace",
    index=False,
    chunksize=10000
)


-78